In [228]:
import glob
import pandas as pd
from pandas import DataFrame as df
from os import path
import csv
from io import StringIO, BytesIO
import numpy as np

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.4/dist-packages/matplotlib/backends/backend_gtk3agg.py:18: UserWarning: The Gtk3Agg backend is known to not work on Python 3.x with pycairo. Try installing cairocffi.
  "The Gtk3Agg backend is known to not work on Python 3.x with pycairo. "


## Merge CSV files

Each cvs file represent a specific word results obtained from the NSF and NIH websites.

In [8]:
def MergeCSVs(files):
    dfs = []
    for file in files:
      word = path.basename(file).split('.')[0]
      text = open(file, "r", encoding="iso-8859-1").read().replace("\r", "\n")
      df_ = df.from_dict(list(csv.DictReader(StringIO(text))))
      df_.insert(0, "word", [word]*len(df_))
      dfs.append(df_)
    return pd.concat(dfs)

In [9]:
NIH_df = MergeCSVs(glob.glob("data/Grants/NIH/*csv"))
NSF_df = MergeCSVs(glob.glob("data/Grants/NSF/*csv"))

In [250]:
NIH_df.head()

,word,ARRA Indicator,Activity,Administering IC,Application ID,Contact PI Person ID,Contact PI / Project Leader,FY,FY Total Cost,FY Total Cost (Sub Projects),IC,Organization Name,Other PI or Project Leader(s),Project Number,Project Title,Serial Number,Subproject Number,Suffix,Support Year,Type
0,bicultural,,R01,NICHD,6521173,1896188,"ALTMAN, JACKI",2002,335137,,HD,UNIVERSITY OF CONNECTICUT STORRS,Not Applicable,5R01HD037515-03,CULTURAL CHANGE IN CHILDREARING BELIEFS AND PR...,37515,,,3,5
1,bicultural,,R01,NICHD,6388079,1896188,"ALTMAN, JACKI",2001,355033,,HD,UNIVERSITY OF CONNECTICUT STORRS,Not Applicable,5R01HD037515-02,CULTURAL CHANGE IN CHILDREARING BELIEFS AND PR...,37515,,,2,5
2,bicultural,,R01,NICHD,6138870,1896188,"ALTMAN, JACKI",2000,415255,,HD,UNIVERSITY OF CONNECTICUT STORRS,Not Applicable,1R01HD037515-01A1,CULTURAL CHANGE IN CHILDREARING BELIEFS AND PR...,37515,,A1,1,1
3,bicultural,,R01,NIMH,7664631,1861565,"AMBADY, NALINI",2009,207082,,MH,TUFTS UNIVERSITY MEDFORD,Not Applicable,5R01MH070833-05,COMMUNICATION OF EMOTION: BEHAVIORAL AND NEURA...,70833,,,5,5
4,bicultural,,R01,NIMH,7488888,1861565,"AMBADY, NALINI",2008,207082,,MH,TUFTS UNIVERSITY MEDFORD,Not Applicable,5R01MH070833-04,COMMUNICATION OF EMOTION: BEHAVIORAL AND NEURA...,70833,,,4,5


In [11]:
NSF_df.head()

,word,ARRAAmount,Abstract,AwardInstrument,AwardNumber,AwardedAmountToDate,Co-PIName(s),EndDate,LastAmendmentDate,NSFDirectorate,...,OrganizationZip,PIEmailAddress,PrincipalInvestigator,Program(s),ProgramElementCode(s),ProgramManager,ProgramReferenceCode(s),StartDate,State,Title
0,bicultural,$0.00,This project is an empirical research study us...,Standard Grant,1251588,"$347,496.00",,08/31/2016,09/16/2013,EHR,...,871199828,pam@aises.org,Pamala Silas,REAL,7625,Jolene K. Jesse,"9150, 8212",09/15/2013,NM,Collaborative Research to Understand the Role ...
1,bicultural,$0.00,This project is an empirical research study us...,Standard Grant,1251516,"$311,981.00",,08/31/2017,09/07/2013,EHR,...,602013149,medin@northwestern.edu,Douglas Medin,REAL,7625,Jolene K. Jesse,8212,09/15/2013,IL,Collaborative Research to Understand the Role ...
2,bicultural,$0.00,This project is an empirical research study us...,Standard Grant,1251532,"$732,512.00",,08/31/2016,09/07/2013,EHR,...,871310001,JPage-Reeves@salud.unm.edu,Janet Page-Reeves,REAL,7625,Jolene K. Jesse,"9150, 9177, SMET, 8212",09/15/2013,NM,Collaborative Research to Understand the Role ...
3,bicultural,$0.00,U.S. surveys of minorities and immigrants allo...,Standard Grant,0751899,"$6,920.00","Robert Groves, Emilia Peytcheva",04/30/2013,05/23/2012,SBE,...,481091274,nschwarz@umich.edu,Norbert Schwarz,"METHOD, MEASURE & STATS",1333,Cheryl L. Eavey,"9179, SMET",05/15/2008,MI,Doctoral Dissertation Research: Language of Ad...
4,bicultural,$0.00,,Standard Grant,8316803,"$261,256.00",,07/31/1987,07/22/1985,EHR,...,860110001,,Gordon Johnson,INSTRUCTIONAL MATERIALS DEVELP,7355,Alice Moses,,02/01/1984,AZ,Outdoor World Science - A Model Bicultural Hig...


In [12]:
!mkdir data/processed

### Save the combined query results

In [51]:
NSF_df.to_csv("data/Grants/processed/nsf_combined.csv", encoding='utf-8')
NIH_df.to_csv("data/Grants/processed/nih_combined.csv", encoding='utf-8')

# Process NIH database

In [80]:
NIH = df.from_csv("data/Grants/processed/nih_combined.csv", encoding='utf-8')

## Load University Database

The goal to map each university that is mentioned in the NIH database to a state. We will rely on two source of information to build a partial mapping:

- University accredation database.
- List of states and their acronyms.

After building the partial list, we will fill the gaps manually.

In [107]:
university_df = pd.DataFrame.from_csv("data/Grants/Accreditation_2015_09.csv")
university_df[["Institution_Name", "Institution_State"]].head()

,Institution_Name,Institution_State
Institution_ID,,
180319,"21st Century Oncology, Inc. School for Radiati...",FL
180319,"21st Century Oncology, Inc. School for Radiati...",FL
180319,"21st Century Oncology, Inc. School for Radiati...",FL
203827,360 Degrees Beauty Academy,TX
211194,4-States Academy of Cosmetology,OK


In [158]:
code_to_state = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [161]:
state_to_code = {state.lower():code for code,state in code_to_state.items()}

In [142]:
university_to_state = {uni.lower():state for uni,state in university_df[["Institution_Name", "Institution_State"]].values}

In [144]:
universities = set(university_to_state.keys())

In [163]:
def FindState(name):
  name = name.lower()
  if name in universities:
    return university_to_state[name]
  for state in state_to_code.keys():
    if state in name:
      return state_to_code[state]
  for n in universities:
    if name in n:
      return university_to_state[n]
  return ""

In [164]:
NIH_universities = {x.lower() for x in set(NIH["Organization Name"].fillna("").values)}

In [179]:
NIH_uni_state_df = pd.DataFrame.from_dict([{"University":uni, "State":FindState(uni)} for uni in NIH_universities])

In [182]:
NIH_uni_state_df.to_csv("data/Grants/processed/university_to_state.csv")

## NIH Modification Steps


1. ~~Remove the following columns: ARRA indicator, Administering IC, Application ID, Contact PI Person ID, Contact PI, Other PI, Project Number, Serial Number, Subproject Number, Suffix~~
1. ~~First column --> give title 'ID #'~~
1. ~~Copy the word data into a new column (title it 'terms')--> code them as the following: 1 = multiculturalism, 2 = polyculturalism, 3 = cultural pluralism, 4 = monocultural, 5 = monoracial, 6 = bicultural, 7 = biracial, 8 = biethnic, 9 = interracial, 10 = multicultural, 11 = multiracial, 12 = polycultural, 13 = polyracial, 14 = polyethnic, 15 = mixed race, 16 = mixed ethnicity, 17 = other race, 18 = other ethnicity~~
1. ~~'Activity' --> Rename 'Grant Program'~~
1. ~~'FY' - 1 --> Rename 'Start Year'~~
1. ~~Combine 'FY Total Cost' and 'FY Total Cost Subproject' Columns --> Rename 'Award Amount'~~
1. ~~'IC' --> Rename 'Grant Organization'~~ 
1. ~~'Organization Name' --> Rename 'Funded Organization'~~
1. Add new column --> 'Organization State' [use csv file in email to get state info for universities in list]
1. ~~'Type'--> Rename 'Grant Type' [Recode as the following: 1 = 1 (New Grant), 2+ = 2 (Continuing Grant)]~~


In [181]:
NIH.loc[:, "FY Total Cost "] = NIH["FY Total Cost "].fillna(0)
NIH.loc[:, "FY Total Cost  (Sub Projects)"] = NIH["FY Total Cost  (Sub Projects)"].fillna(0)
NIH["Award Amount"] = NIH["FY Total Cost  (Sub Projects)"] + NIH["FY Total Cost "]

remaining_cols = [c for c in NIH.columns if c not in
                  {'ARRA Indicator', 'Administering  IC', 'Application ID', 'Contact PI  Person ID',
                   'Other PI or Project Leader(s)', 'Project Number', 'Serial Number', 'Subproject Number',
                   'Suffix', "FY Total Cost ", "FY Total Cost  (Sub Projects)"}]
NIH_minimal = NIH[remaining_cols]
NIH_minimal.index.rename("ID", inplace=True)
NIH_minimal = NIH_minimal.rename(columns={"word": "Term",
                                          "Activity": "Grant Program",
                                          "FY": "Start Year",
                                          "Type": "Grant Type",
                                          "Grant Organization": "Funded Organization"})
NIH_minimal = NIH_minimal.rename(columns={"IC": "Grant Organization"})
id_to_term = { 1: "multiculturalism", 2: "polyculturalism", 3: "cultural pluralism", 4: "monocultural",
              5: "monoracial", 6: "bicultural", 7: "biracial", 8: "biethnic", 9: "interracial",
              10: "multicultural", 11: "multiracial", 12: "polycultural", 13: "polyracial",
              14: "polyethnic", 15: "mixed race", 16: "mixed ethnicity", 17: "other race", 18: "other ethnicity"}
term_to_id = {term:id for id, term in id_to_term.items()}
NIH_minimal.loc[:, "Term"] = NIH_minimal.Term.map(term_to_id)
NIH_minimal.loc[:, "Grant Type"] = NIH_minimal["Grant Type"].map(lambda x: 1 if x=='1' else 2)
NIH_minimal.head()

,Term,Grant Program,Contact PI / Project Leader,Start Year,Grant Organization,Organization Name,Project Title,Subproject Number,Support Year,Grant Type,Award Amount
ID,,,,,,,,,,,
0,6,R01,"ALTMAN, JACKI",2002,HD,UNIVERSITY OF CONNECTICUT STORRS,CULTURAL CHANGE IN CHILDREARING BELIEFS AND PR...,NaN,3,2,335137
1,6,R01,"ALTMAN, JACKI",2001,HD,UNIVERSITY OF CONNECTICUT STORRS,CULTURAL CHANGE IN CHILDREARING BELIEFS AND PR...,NaN,2,2,355033
2,6,R01,"ALTMAN, JACKI",2000,HD,UNIVERSITY OF CONNECTICUT STORRS,CULTURAL CHANGE IN CHILDREARING BELIEFS AND PR...,NaN,1,1,415255
3,6,R01,"AMBADY, NALINI",2009,MH,TUFTS UNIVERSITY MEDFORD,COMMUNICATION OF EMOTION: BEHAVIORAL AND NEURA...,NaN,5,2,207082
4,6,R01,"AMBADY, NALINI",2008,MH,TUFTS UNIVERSITY MEDFORD,COMMUNICATION OF EMOTION: BEHAVIORAL AND NEURA...,NaN,4,2,207082


#### Save new NIH dataset

In [53]:
NIH_minimal.to_csv("data/Grants/processed/nih_clean.csv", encoding='utf-8')

# Process NSF Database

In [ ]:
NSF = df.from_csv("data/Grants/processed/nsf_combined.csv", encoding='utf-8')

## NSF Modification Steps 

1. ~~Remove the following columns: ARRA amount, Last Amendment Date, Organization City, Organization Zip, Program Element Code, State~~
1. ~~First column --> give title 'ID #'~~
1. ~~Copy the word data into a new column (title it 'terms')--> code them as the following: 1 = multiculturalism, 2 = polyculturalism, 3 = cultural pluralism, 4 = monocultural, 5 = monoracial, 6 = bicultural, 7 = biracial, 8 = biethnic, 9 = interracial, 10 = multicultural, 11 = multiracial, 12 = polycultural, 13 = polyracial, 14 = polyethnic, 15 = mixed race, 16 = mixed ethnicity, 17 = other race, 18 = other ethnicity~~
1. ~~'Abstract' --> Rename 'Project Abstract'~~
1. 'Award Instrument'--> Rename 'Grant Type' [Recode as the following: 1 = Standard Grant, 2 = Continuing Grant, 3 = Fellowship, ...] ***we should speak about this to see all possible responses in this column***
1. ~~'Awarded Amount to Date' --> Rename 'Award Amount'~~
1. 'NSF Directorate' OR 'NSF Organization' --> Rename 'Grant Organization' ***We should discuss this before you execute this***
1. ~~'Organization Name' --> Rename 'Funded Organization'~~
1. 'Program(s)' --> Rename 'Program Topic' [Recode --> first we must have a list of all possible responses in dataset]
1. ~~'Start Date' --> Rename 'Start Year' [Only use year from date information]~~
1. ~~New Column 'Support Year': 2015-Start Year~~ (I do not understand this one!)
1. ~~New Column 'Award Length': End Date - Start Year (Years only)~~
1. ~~Title --> Rename 'Project Title'~~

In [187]:
NSF["AwardInstrument"].value_counts()

Standard Grant                    2395
Continuing grant                   742
Fellowship                          40
Cooperative Agreement               31
Interagency Agreement                4
Fixed Price Award                    3
Contract Interagency Agreement       2
Contract                             1
Name: AwardInstrument, dtype: int64

In [203]:
NSF["Program(s)"].fillna("").apply(lambda x: x.strip()[:80]).value_counts()

SOCIOLOGY                                                                           307
CULTURAL ANTHROPOLOGY                                                               135
POLITICAL SCIENCE                                                                   130
LAW AND SOCIAL SCIENCES                                                             100
SOCIAL PSYCHOLOGY                                                                    90
GEOGRAPHY AND SPATIAL SCIENCES                                                       89
RES ON GENDER IN SCI & ENGINE                                                        75
ECONOMICS                                                                            73
SOCIOLOGY, LAW AND SOCIAL SCIENCES                                                   43
Hist & Philosophy of SET                                                             38
                                                                                     37
ARCHAEOLOGY                     

In [247]:
NSF.loc[:, "Support Year"] = 2015 - pd.DatetimeIndex(NSF["StartDate"]).year
NSF.loc[:, "Award Length"] = pd.DatetimeIndex(NSF["EndDate"]).year - pd.DatetimeIndex(NSF["StartDate"]).year
NSF.loc[:, "Start Year"] = pd.DatetimeIndex(NSF["StartDate"]).year
remaining_cols = [c for c in NSF.columns if c not in
                  {"ARRAAmount", "LastAmmendmentDate", "OrganizationCity", "OrganizationZip",
                   "ProgramElementCode", "StartDate"}]
NSF_minimal = NSF[remaining_cols]
NSF_minimal.index.rename("ID", inplace=True)
NSF_minimal = NSF_minimal.rename(columns={"word": "Term", "Abstract": "Project Abstract",
                                         "AwardedAmountToDate": "Award Amount",
                                          "OrganizationName": "Funded Organization", "Title":"Project Title",
                                         "StartDate":"Start Year"})
id_to_term = { 1: "multiculturalism", 2: "polyculturalism", 3: "cultural pluralism", 4: "monocultural",
              5: "monoracial", 6: "bicultural", 7: "biracial", 8: "biethnic", 9: "interracial",
              10: "multicultural", 11: "multiracial", 12: "polycultural", 13: "polyracial",
              14: "polyethnic", 15: "mixed race", 16: "mixed ethnicity", 17: "other race", 18: "other ethnicity"}
term_to_id = {term:id for id, term in id_to_term.items()}
NSF_minimal.loc[:, "Term"] = NSF_minimal.Term.map(term_to_id)
NSF_minimal.head()

,Term,Project Abstract,AwardInstrument,AwardNumber,Award Amount,Co-PIName(s),EndDate,LastAmendmentDate,NSFDirectorate,NSFOrganization,...,PrincipalInvestigator,Program(s),ProgramElementCode(s),ProgramManager,ProgramReferenceCode(s),State,Project Title,Support Year,Award Length,Start Year
ID,,,,,,,,,,,,,,,,,,,,,
0,6,This project is an empirical research study us...,Standard Grant,1251588,"$347,496.00",NaN,08/31/2016,09/16/2013,EHR,DRL,...,Pamala Silas,REAL,7625,Jolene K. Jesse,"9150, 8212",NM,Collaborative Research to Understand the Role ...,2,3,2013
1,6,This project is an empirical research study us...,Standard Grant,1251516,"$311,981.00",NaN,08/31/2017,09/07/2013,EHR,DRL,...,Douglas Medin,REAL,7625,Jolene K. Jesse,8212,IL,Collaborative Research to Understand the Role ...,2,4,2013
2,6,This project is an empirical research study us...,Standard Grant,1251532,"$732,512.00",NaN,08/31/2016,09/07/2013,EHR,DRL,...,Janet Page-Reeves,REAL,7625,Jolene K. Jesse,"9150, 9177, SMET, 8212",NM,Collaborative Research to Understand the Role ...,2,3,2013
3,6,U.S. surveys of minorities and immigrants allo...,Standard Grant,0751899,"$6,920.00","Robert Groves, Emilia Peytcheva",04/30/2013,05/23/2012,SBE,SES,...,Norbert Schwarz,"METHOD, MEASURE & STATS",1333,Cheryl L. Eavey,"9179, SMET",MI,Doctoral Dissertation Research: Language of Ad...,7,5,2008
4,6,NaN,Standard Grant,8316803,"$261,256.00",NaN,07/31/1987,07/22/1985,EHR,DRL,...,Gordon Johnson,INSTRUCTIONAL MATERIALS DEVELP,7355,Alice Moses,NaN,AZ,Outdoor World Science - A Model Bicultural Hig...,31,3,1984


#### Save results

In [248]:
NSF_minimal.to_csv("data/Grants/processed/nsf_clean.csv", encoding='utf-8')

Mega NIH/NSF Dataset 
--------------------

1. New Column: 'Grant Agency' [Code --> 1 = NIH, 2 = NSF]
1. Combine the two datasets (after following the instructions below)

***NIH***

Add NIH to the beginning of each remaining column

Remove the following columns:

1. Grant Program
1. Grant Organization
1. Funded Organization?

***NSF***

Add NSF to the beginning of each remaining column

Remove the following columns: 

1. Abstract
1. NSF Organization/NSF Directorate
1. Funded Organization?
1. Program Topic 